In [1]:
import os
import pandas as pd
import subprocess
import pandas as pd
from collections import defaultdict
from pyswarm import pso

# ----------------- CONFIGURACIÓN -----------------
INPUT_CSV     = 'OnlineRetail_Categorias.csv'    # dataset
SPMF_TXT      = 'Avance/ItemSetsAvancePreliminar/transactions.txt'  # Archivo que SPMF va a leer
HUIM_OUTPUT   = 'Avance/PatronesHUIM/Patrones_HUIM.txt'   # Salida de HUIM
SELECTED_OUT  = 'Avance/PatronesPSO/PatronesSelecionados.txt'  # Patrones elegidos por PSO
SPMF_JAR_DIR  = '/home/daxtiniyni/tools/spmf'    # ruta al spmf.jar
# --------------------------------------------------

In [ ]:
def convert_csv_to_spmf(csv_path, txt_path):

    # 2.1 Carga el CSV
    df = pd.read_csv(csv_path, encoding='latin1')
    
    # 2.2 Calcula la utilidad de cada fila: Quantity * UnitPrice
    df['Utilidad'] = df['Quantity'] * df['UnitPrice']
    
    # 2.3 Crea un mapeo de categorías a enteros (Category --> CatID)
    únicas = df['Categoria'].unique()
    cat2id = {cat: idx+1 for idx, cat in enumerate(únicas)}  # empiezo IDs desde 1
    df['CatID'] = df['Categoria'].map(cat2id)
    
    # 2.4 Agrupa por cada transacción y categoría: suma utilidades de esa categoría en esa factura
    grouped = df.groupby(['InvoiceNo', 'CatID'])['Utilidad'].sum().reset_index()
    
    # 2.5 Ahora agrupo SOLO por transacción (InvoiceNo) para armar cada línea de SPMF
    txs = grouped.groupby('InvoiceNo')
    
    # 2.6 Creo carpeta si no existe
    os.makedirs(os.path.dirname(txt_path), exist_ok=True)
    
    total_utils = []  # para cada factura, guardo la utilidad total
    with open(txt_path, 'w') as f:
        for invoice, group in txs:
            # Para cada categoría dentro de la factura, hacemos (CatID, utilidad_decimal)
            items = list(group['CatID'])
            # Multiplico por 100 y convierto a entero para evitar decimales en SPMF
            utils = list((group['Utilidad'] * 100).astype(int))
            # Ordeno por CatID para que SPMF esté feliz
            sorted_pairs = sorted(zip(items, utils), key=lambda x: x[0])
            
            items_sorted = [str(item) for item, _ in sorted_pairs]
            utils_sorted = [str(u) for _, u in sorted_pairs]
            total_util = sum(int(u) for u in utils_sorted)
            total_utils.append(total_util)
            
            # Formato: "id1 id2 ... idN : total_util : u1 u2 ... uN"
            linea = " ".join(items_sorted) + ":" + str(total_util) + ":" + " ".join(utils_sorted) + "\n"
            f.write(linea)
    
    print(f"[+] BLOQUE 1 completado: CSV → SPMF ({len(txs)} transacciones).")
    return cat2id, total_utils


In [3]:
def run_huim(input_path, output_path, min_util):

    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    jar = os.path.join(SPMF_JAR_DIR, 'spmf.jar')
    if not os.path.isfile(jar):
        raise FileNotFoundError(f"spmf.jar no encontrado en {SPMF_JAR_DIR}")
    
    cmd = [
        'java', '-jar', jar,
        'run', 'HUI-Miner',
        input_path,
        output_path,
        str(min_util)
    ]
    try:
        salida = subprocess.check_output(cmd, stderr=subprocess.STDOUT)
        print(f"[+] BLOQUE 2 completado: HUIM ejecutado con min_util={min_util}.\n{salida.decode()}")
    except subprocess.CalledProcessError as e:
        print(f"[!] Error en HUIM:\n{e.output.decode()}")
        raise

In [ ]:
def parse_patterns(path):
    
    patterns = []
    with open(path, 'r') as f:
        for linea in f:
            if not linea.strip():
                continue
            partes = linea.strip().split('#UTIL:')
            items = [int(tok) for tok in partes[0].split()]
            utilidad = int(partes[1])
            patterns.append({'items': items, 'utility': utilidad})
    print(f"[+] BLOQUE 2 completado: Cargados {len(patterns)} patrones HUIM.")
    return patterns

In [5]:
def save_selected(selected, output_path):
    
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, 'w') as f:
        for p in selected:
            linea = " ".join(map(str, p['items'])) + "#UTIL:" + str(p['utility']) + "\n"
            f.write(linea)
    print(f"[+] BLOQUE 5 completado: Patrones guardados en {output_path}.")

In [ ]:
def construir_mapeo_categorias(csv_path):
 
    df = pd.read_csv(csv_path, dtype=str, encoding='latin1', low_memory=False)
    categorias_unicas = sorted(df['Categoria'].unique())
    # Asignamos IDs del 1 al N, pero como strings. Ej: 'Almacenamiento': '1'
    mapeo = { cat: str(i+1) for i, cat in enumerate(categorias_unicas) }
    return mapeo

In [ ]:
def cargar_transacciones(csv_path, mapeo_cat):

    df = pd.read_csv(csv_path, dtype={'InvoiceNo': str}, encoding='latin1', low_memory=False)
    transacciones = defaultdict(set)
    for _, row in df.iterrows():
        cat_texto = row['Categoria']
        cat_id = mapeo_cat.get(cat_texto)
        # Si por alguna razón falta la categoría en el mapeo, la ignoramos
        if cat_id:
            transacciones[row['InvoiceNo']].add(cat_id)
    return transacciones

In [ ]:
def cargar_patrones_huim(file_path):
    patrones = []
    with open(file_path, 'r', encoding='latin1') as f:
        for linea in f:
            linea = linea.strip()
            if not linea:
                continue
            # Quedarse solo con la parte antes de "#UTIL"
            partes = linea.split('#UTIL')[0].strip()
            # Las categorías van separadas por espacio
            lista_ids = partes.split()
            # Normalizamos como tupla de strings (sin strings vacíos)
            pat_tuple = tuple([item for item in lista_ids if item])
            if pat_tuple:
                patrones.append(pat_tuple)
    return patrones

In [ ]:
def freq_booleana(patron, transacciones):
    cnt = 0
    for items in transacciones.values():
        # Si cada ID de patron está en el set de items de la transacción
        if all(i in items for i in patron):
            cnt += 1
    return cnt

In [ ]:
def extraer_psf(transacciones, patrones_huim, min_freq):
    psf = {}
    for pat in patrones_huim:
        f = freq_booleana(pat, transacciones)
        if f >= min_freq:
            psf[pat] = f
    return psf

In [11]:
if __name__ == '__main__':
    # BLOQUE 1: Conversiones y HUIM
    
    # 6.1) Conversiones y percentiles
    cat2id, total_utils = convert_csv_to_spmf(INPUT_CSV, SPMF_TXT)
    
    # 6.2) Definir min_util como el percentil 75 de las utilidades totales
    serie = pd.Series(total_utils)
    min_util = int(serie.quantile(0.6))
    print(f"[+] Umbral MIN_UTIL calculado: {min_util}.\n")
    
    # 6.3) Ejecutar HUI-Miner de SPMF
    run_huim(SPMF_TXT, HUIM_OUTPUT, min_util)
    
    # 6.4) Parsear patrones HUIM
    patterns = parse_patterns(HUIM_OUTPUT)
    
    # 6.5) Contexto final: mostramos el mapeo de categorías por si lo quieres traducir
    print("\n¡Proceso completo!")
    print("Mapa de categorías (Categoría → CatID):")
    for cat, idx in cat2id.items():
        print(f"  {idx} → {cat}")
    
    
    # BLOQUE 2: Mapeo, transacciones y Patrones Similares

    CSV_PATH = 'OnlineRetail_Categorias.csv'
    HUIM_FILE = 'Avance/PatronesHUIM/Patrones_HUIM.txt'
    MIN_FREQ = 3826  # Mínimo de transacciones donde debe aparecer el patrón

    # 13.1) Construimos el mapeo categoría→ID
    mapeo_cat = construir_mapeo_categorias(CSV_PATH)
    print("\nMapeo categoría → ID numérico (string):")
    for cat, idx in mapeo_cat.items():
        print(f"   '{cat}' → '{idx}'")
    print()

    # 13.2) Cargo transacciones con IDs en vez de nombres
    trans = cargar_transacciones(CSV_PATH, mapeo_cat)
    print(f"⚡ Cargadas {len(trans)} facturas con IDs de categorías.\n")

    # 13.3) Cargo patterns HUIM (tuplas de strings numéricos)
    patrones_huim = cargar_patrones_huim(HUIM_FILE)
    print(f"Cargados {len(patrones_huim)} patrones HUIM desde '{HUIM_FILE}'.\n")

    # 13.4) Extraigo Patrones Similares Frecuentes
    psf = extraer_psf(trans, patrones_huim, MIN_FREQ)

    # 13.5) Muestro resultados
    print(f"Patrones Similares Frecuentes (freq >= {MIN_FREQ}): {len(psf)} encontrados.\n")
    if psf:
        # Ordeno por largo de patrón (desc) y luego por frecuencia (desc)
        for pat, freq in sorted(psf.items(), key=lambda x: (-len(x[0]), -x[1])):
            print(f"   {','.join(pat)}  → {freq} facturas")
    else:
        print("(No se encontraron, verifica el MIN_FREQ.)")

    # 13.6) Si quieres guardarlos en un archivo de texto:
    output_dir = 'Avance/PatronesSimilares'
    os.makedirs(output_dir, exist_ok=True)  # ← Aseguramos que exista la carpeta
    output_path = os.path.join(output_dir, 'psf_desde_huim.txt')
    
    with open(output_path, 'w', encoding='utf-8') as f_out:
        for pat, freq in sorted(psf.items(), key=lambda x: (-len(x[0]), -x[1])):
            f_out.write(f"{','.join(pat)} : {freq}\n")
    print(f"\n  Resultado escrito en '{output_path}' (si psf no está vacío).")



/tmp/ipykernel_11407/3092087419.py:4: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path, encoding='latin1')


[+] BLOQUE 1 completado: CSV → SPMF (19962 transacciones).
[+] Umbral MIN_UTIL calculado: 34855.

[+] BLOQUE 2 completado: HUIM ejecutado con min_util=34855.
>/home/daxtiniyni/tools/spmf/spmf.jar
=============  HUI-MINER ALGORITHM - STATS =============
 Total time ~ 408 ms
 Memory ~ 148.09893035888672 MB
 High-utility itemsets count : 8191
 Join count : 8178

[+] BLOQUE 3 completado: Cargados 8191 patrones HUIM.

¡Proceso completo!
Mapa de categorías (Categoría → CatID):
  1 → Hogar & DecoraciÃ³n
  2 → Otros
  3 → Vajilla & CristalerÃ­a
  4 → Juguetes & NiÃ±os
  5 → Almacenamiento
  6 → Bakeware
  7 → PapelerÃ­a & Regalos
  8 → IluminaciÃ³n
  9 → Fiestas & Temporada
  10 → Bolsas
  11 → Cocina & Utensilios
  12 → Textiles & Accesorios
  13 → Muebles & Accesorios

Mapeo categoría → ID numérico (string):
   'Almacenamiento' → '1'
   'Bakeware' → '2'
   'Bolsas' → '3'
   'Cocina & Utensilios' → '4'
   'Fiestas & Temporada' → '5'
   'Hogar & DecoraciÃ³n' → '6'
   'IluminaciÃ³n' → '7'
   'J

In [12]:
import os
import pandas as pd
from collections import defaultdict

# 1) Ruta de archivos
CSV_PATH    = 'OnlineRetail_Categorias.csv'
PSF_TXT     = 'Avance/PatronesSimilares/psf_desde_huim.txt'
OUTPUT_CSV  = 'Avance/Resultado/psf_convertido.csv'

# 2) Reconstruir el mapeo categoría → ID (string) y luego invertirlo a ID → categoría
def construir_mapeo_categorias(csv_path):
    df = pd.read_csv(csv_path, dtype=str, encoding='latin1', low_memory=False)
    categorias_unicas = sorted(df['Categoria'].unique())
    # Mapeo categoría → ID (string)
    mapeo = {cat: str(i+1) for i, cat in enumerate(categorias_unicas)}
    return mapeo

mapeo_cat = construir_mapeo_categorias(CSV_PATH)
id2cat = {vid: cat for cat, vid in mapeo_cat.items()}

# 3) Leer psf_desde_huim.txt y convertir cada línea de IDs a nombres
filas = []
with open(PSF_TXT, 'r', encoding='utf-8') as f:
    for linea in f:
        linea = linea.strip()
        if not linea:
            continue
        # Formato esperado: "id1,id2,... : freq"
        partes = linea.split(':')
        ids_str = partes[0].strip()       # "id1,id2,..."
        freq = int(partes[1].strip())     # frecuencia como entero
        lista_ids = [x.strip() for x in ids_str.split(',') if x.strip()]
        # Convertir cada ID a nombre de categoría
        lista_cats = [id2cat.get(x, f"ID_{x}_desconocido") for x in lista_ids]
        pattern_nombres = ", ".join(lista_cats)
        filas.append({
            'Pattern_Categorias': pattern_nombres,
            'Frecuencia': freq
        })

# 4) Crear carpeta de salida si no existe
out_dir = os.path.dirname(OUTPUT_CSV)
os.makedirs(out_dir, exist_ok=True)

# 5) Generar DataFrame y guardar a CSV
df_out = pd.DataFrame(filas, columns=['Pattern_Categorias', 'Frecuencia'])
df_out.to_csv(OUTPUT_CSV, index=False, encoding='utf-8')

print(f"psf convertido guardado en '{OUTPUT_CSV}'.")


psf convertido guardado en 'Avance/Resultado/psf_convertido.csv'.


In [13]:
import numpy as np
import pandas as pd
import os

# Cargar DataFrame de patrones similares frecuentes
df = pd.read_csv('Avance/Resultado/psf_convertido.csv')
df = df.rename(columns={df.columns[0]: 'pattern', df.columns[1]: 'utility'})

# Preparar datos
patterns = df['pattern'].astype(str).apply(lambda x: x.split(',')).tolist()
U = df['utility'].values

# Matriz de similitud (Jaccard)
n = len(patterns)
S = np.zeros((n, n))
for i in range(n):
    set_i = set(patterns[i])
    for j in range(n):
        set_j = set(patterns[j])
        union = set_i | set_j
        S[i, j] = len(set_i & set_j) / len(union) if union else 0
np.fill_diagonal(S, 0.0)

# Función objetivo
def fitness(x, U, S, lambda_param):
    util_sum = np.dot(x, U)
    redundancy = (x @ S @ x) / 2
    return util_sum - lambda_param * redundancy

# Parámetros de PSO
num_particles = 30
num_iterations = 100
lambda_param = 0.5
w = 0.72
c1 = c2 = 1.49

# Inicialización
dim = n
velocities = np.random.uniform(-1, 1, (num_particles, dim))
positions = np.random.choice([0, 1], size=(num_particles, dim))

pbest_positions = positions.copy()
pbest_scores = np.array([fitness(p, U, S, lambda_param) for p in positions])

gbest_idx = np.argmax(pbest_scores)
gbest_position = pbest_positions[gbest_idx].copy()
gbest_score = pbest_scores[gbest_idx]

# PSO
for it in range(num_iterations):
    for i in range(num_particles):
        r1, r2 = np.random.rand(dim), np.random.rand(dim)
        velocities[i] = (
            w * velocities[i]
            + c1 * r1 * (pbest_positions[i] - positions[i])
            + c2 * r2 * (gbest_position - positions[i])
        )
        probs = 1.0 / (1.0 + np.exp(-velocities[i]))
        positions[i] = (np.random.rand(dim) < probs).astype(int)

        score = fitness(positions[i], U, S, lambda_param)
        if score > pbest_scores[i]:
            pbest_positions[i] = positions[i].copy()
            pbest_scores[i] = score
            if score > gbest_score:
                gbest_position = positions[i].copy()
                gbest_score = score

# Resultados
selected_idxs = np.where(gbest_position == 1)[0]
results_df = pd.DataFrame({
    'pattern': df['pattern'].iloc[selected_idxs].values,
    'utility': df['utility'].iloc[selected_idxs].values
})

# Crear directorio y guardar resultados
output_dir = 'Avance/Resultado'
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, 'Patrones_PSO.csv')
results_df.to_csv(output_path, index=False)

print(f"Guardados {len(selected_idxs)} patrones en: {output_path}")


Guardados 50 patrones en: Avance/Resultado/Patrones_PSO.csv
